In [ ]:
import config
import pyspark

In [ ]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [ ]:
df = ( spark.read
    .format('mongodb')
    .option('database'      , f'{config.MongoDb.database}')
    .option('collection'    , f'{config.MongoDb.collection}')
    .option('connection.uri', f'mongodb://{config.MongoDb.user}:{config.MongoDb.password}@{config.MongoDb.host}:{config.MongoDb.port}')
	.load()
)
df.show(10)

In [ ]:
df.write.format('csv').save('sentiments.csv')

In [ ]:
import streamlit as st
import pandas as pd
import altair as alt
import plotly.express as px

In [ ]:
st.set_page_config(
    page_title="Sentiment Dashboard",
    page_icon ="🏂",
    layout    ="wide",
    initial_sidebar_state= "expanded"
)

alt.themes.enable("dark")